In [1]:
!pip install selenium

   ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/10.5 MB 2.2 MB/s eta 0:00:05
   - -------------------------------------- 0.5/10.5 MB 5.3 MB/s eta 0:00:02
   ---- ----------------------------------- 1.2/10.5 MB 8.8 MB/s eta 0:00:02
   ------- -------------------------------- 2.0/10.5 MB 10.6 MB/s eta 0:00:01
   ----------- ---------------------------- 2.9/10.5 MB 12.3 MB/s eta 0:00:01
   -------------- ------------------------- 3.8/10.5 MB 13.4 MB/s eta 0:00:01
   ------------------ --------------------- 4.9/10.5 MB 15.0 MB/s eta 0:00:01
   ----------------------- ---------------- 6.1/10.5 MB 16.2 MB/s eta 0:00:01
   --------------------------- ------------ 7.3/10.5 MB 17.4 MB/s eta 0:00:01
   -------------------------------- ------- 8.7/10.5 MB 18.5 MB/s eta 0:00:01
   ------------------------------------- -- 9.9/10.5 MB 19.8 MB/s eta 0:00:01
   ---------------------------------------- 10.5/10.5 MB 21.1 MB/s eta 0:00

In [6]:
#Import relevant libraries 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import Edge
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options as EdgeOptions

import re
import time
from datetime import datetime
import pandas as pd

# Capturing Steam Reviews

## Craftopia Steam Reviews

In [24]:
# Get the steam game id and url for the review + filter

game_id = 1307550 #change the id for each game, in this case, id is for a game called [Craftopia]
url_template = "https://steamcommunity.com/app/{}/reviews/?p=1&browsefilter=mostrecent&filterLanguage=english"
url = url_template.format(game_id)

print(url)

https://steamcommunity.com/app/1307550/reviews/?p=1&browsefilter=mostrecent&filterLanguage=english


In [25]:
# Initialize Edge Options, must already have edgedriver application

options = EdgeOptions()
options.use_chromium = True  # Specify using Chromium-based Edge

language = "en-US"
options.add_argument(f"--lang={language}")

# Initialize Edge WebDriver with options
driver = Edge(options=options)

driver.maximize_window()
driver.get(url)

In [26]:
# Function needed to get steam reviews

def get_current_scroll_position(driver):
    return driver.execute_script("return window.pageYOffset;")

def scroll_to_bottom(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)

def get_steam_id(card):
    profile_url = card.find_element(By.XPATH, './/div[@class="apphub_friend_block"]/div/a[2]').get_attribute('href')
    steam_id = profile_url.split('/')[-2]
    return steam_id

def scrape_review_data(card):
    date_posted_element = card.find_element(By.XPATH, './/div[@class="apphub_CardTextContent"]/div[@class="date_posted"]')
    date_posted = date_posted_element.text.strip()
    
    try:
        received_compensation_element = card.find_element(By.CLASS_NAME, "received_compensation").text
    except NoSuchElementException:
        received_compensation_element = ""

    card_text_content_element = card.find_element(By.CLASS_NAME, "apphub_CardTextContent")
    review_content = card_text_content_element.text.strip()
    excluded_elements = [date_posted, received_compensation_element] 

    for excluded_element in excluded_elements:
        review_content = review_content.replace(excluded_element, "")
    review_content = review_content.replace("\n", "")

    review_length = len(review_content.replace(" ", ""))

    thumb_text = card.find_element(By.XPATH, './/div[@class="reviewInfo"]/div[2]').text
    play_hours = card.find_element(By.XPATH, './/div[@class="reviewInfo"]/div[3]').text

    return review_content, thumb_text, review_length, play_hours, date_posted

In [27]:
reviews = []
steam_ids_set = set()
max_scroll_attempts = 5
desired_review_count = 300
current_review_count = 0

try:
    last_position = get_current_scroll_position(driver)
    running = True
    while running:
        cards = driver.find_elements(By.CLASS_NAME, "apphub_Card")
        
        for card in cards[-20:]:  # Process the last 20 cards
            steam_id = get_steam_id(card)
            if steam_id in steam_ids_set:
                continue
            else:
                review = scrape_review_data(card)
                reviews.append(review)
                current_review_count += 1
                if current_review_count >= desired_review_count:
                    running = False
                    break
        
        if not running:
            break
        
        scroll_attempt = 0
        while scroll_attempt < max_scroll_attempts:
            scroll_to_bottom(driver)
            curr_position = get_current_scroll_position(driver)
            
            if curr_position == last_position:
                scroll_attempt += 1
                time.sleep(3)
                
                if curr_position >= 3:
                    running = False
                    break
            else:
                last_position = curr_position
                break
    
except Exception as e:
    print(e)

finally:
    driver.quit() 

In [28]:
df = pd.DataFrame(reviews, columns= ['ReviewText', 'Review', 'ReviewLength', 'PlayHours', 'DatePosted'])

In [29]:
df

,ReviewText,Review,ReviewLength,PlayHours,DatePosted
0,EARLY ACCESS REVIEWIt's cool,Recommended,25,112.3 hrs on record,Posted: 4 April
1,EARLY ACCESS REVIEWI love the references and m...,Recommended,132,73.7 hrs on record,Posted: 3 April
2,"EARLY ACCESS REVIEWit's like taking zelda, min...",Recommended,85,6.8 hrs on record,Posted: 2 April
3,EARLY ACCESS REVIEWfun,Recommended,20,12.1 hrs on record,Posted: 1 April
4,"EARLY ACCESS REVIEWThis game is fun, and engag...",Recommended,87,22.3 hrs on record,Posted: 1 April
...,...,...,...,...,...
295,EARLY ACCESS REVIEWive been playing this game ...,Recommended,516,235.0 hrs on record,Posted: January 26
296,EARLY ACCESS REVIEWrest well my sweet prince. ...,Not Recommended,441,8.5 hrs on record,Posted: January 25
297,EARLY ACCESS REVIEWYa'll haven't heard of Scra...,Recommended,341,12.1 hrs on record,Posted: January 25
298,EARLY ACCESS REVIEWI would like to start this ...,Not Recommended,1782,224.2 hrs on record,Posted: January 25


In [30]:
df.to_csv('Craftopia.csv', index=False)

## Palworld Steam Reviews

In [21]:
url = url_template.format(1623730)

# Initialize Edge Options, must already have edgedriver application
options = EdgeOptions()
options.use_chromium = True  # Specify using Chromium-based Edge

language = "en-US"
options.add_argument(f"--lang={language}")

# Initialize Edge WebDriver with options
driver = Edge(options=options)

driver.maximize_window()
driver.get(url)

reviews = []
steam_ids_set = set()
max_scroll_attempts = 5
desired_review_count = 300
current_review_count = 0

try:
    last_position = get_current_scroll_position(driver)
    running = True
    while running:
        cards = driver.find_elements(By.CLASS_NAME, "apphub_Card")
        
        for card in cards[-20:]:  # Process the last 20 cards
            steam_id = get_steam_id(card)
            if steam_id in steam_ids_set:
                continue
            else:
                review = scrape_review_data(card)
                reviews.append(review)
                current_review_count += 1
                if current_review_count >= desired_review_count:
                    running = False
                    break
        
        if not running:
            break
        
        scroll_attempt = 0
        while scroll_attempt < max_scroll_attempts:
            scroll_to_bottom(driver)
            curr_position = get_current_scroll_position(driver)
            
            if curr_position == last_position:
                scroll_attempt += 1
                time.sleep(3)
                
                if curr_position >= 3:
                    running = False
                    break
            else:
                last_position = curr_position
                break
    
except Exception as e:
    print(e)

finally:
    driver.quit() 

In [22]:
df1 = pd.DataFrame(reviews, columns= ['ReviewText', 'Review', 'ReviewLength', 'PlayHours', 'DatePosted'])
df1

,ReviewText,Review,ReviewLength,PlayHours,DatePosted
0,EARLY ACCESS REVIEWi like,Recommended,22,25.8 hrs on record,Posted: 5 April
1,EARLY ACCESS REVIEWPokemon but better,Recommended,33,10.1 hrs on record,Posted: 5 April
2,EARLY ACCESS REVIEWgood game,Recommended,25,17.8 hrs on record,Posted: 5 April
3,EARLY ACCESS REVIEWYou can kidnap people :),Recommended,37,5.0 hrs on record,Posted: 5 April
4,EARLY ACCESS REVIEWFun game.,Recommended,25,122.1 hrs on record,Posted: 5 April
...,...,...,...,...,...
295,EARLY ACCESS REVIEWslavery like the good oll days,Recommended,42,40.6 hrs on record,Posted: April 5
296,EARLY ACCESS REVIEWCommitted crimes against hu...,Recommended,47,27.3 hrs on record,Posted: April 5
297,EARLY ACCESS REVIEWnfgadcmhzgdsifvghgtdscjydsg...,Recommended,52,176.3 hrs on record,Posted: April 5
298,"EARLY ACCESS REVIEWI love this game, and I spe...",Recommended,191,55.3 hrs on record,Posted: April 5


In [23]:
df1.to_csv('Palworld.csv', index=False)

## Lethal Company Steam Reviews

In [31]:
url = url_template.format(1966720)

# Initialize Edge Options, must already have edgedriver application
options = EdgeOptions()
options.use_chromium = True  # Specify using Chromium-based Edge

language = "en-US"
options.add_argument(f"--lang={language}")

# Initialize Edge WebDriver with options
driver = Edge(options=options)

driver.maximize_window()
driver.get(url)

reviews = []
steam_ids_set = set()
max_scroll_attempts = 5
desired_review_count = 300
current_review_count = 0

try:
    last_position = get_current_scroll_position(driver)
    running = True
    while running:
        cards = driver.find_elements(By.CLASS_NAME, "apphub_Card")
        
        for card in cards[-20:]:  # Process the last 20 cards
            steam_id = get_steam_id(card)
            if steam_id in steam_ids_set:
                continue
            else:
                review = scrape_review_data(card)
                reviews.append(review)
                current_review_count += 1
                if current_review_count >= desired_review_count:
                    running = False
                    break
        
        if not running:
            break
        
        scroll_attempt = 0
        while scroll_attempt < max_scroll_attempts:
            scroll_to_bottom(driver)
            curr_position = get_current_scroll_position(driver)
            
            if curr_position == last_position:
                scroll_attempt += 1
                time.sleep(3)
                
                if curr_position >= 3:
                    running = False
                    break
            else:
                last_position = curr_position
                break
    
except Exception as e:
    print(e)

finally:
    driver.quit() 

In [32]:
df2 = pd.DataFrame(reviews, columns= ['ReviewText', 'Review', 'ReviewLength', 'PlayHours', 'DatePosted'])
df2

,ReviewText,Review,ReviewLength,PlayHours,DatePosted
0,EARLY ACCESS REVIEWSimple game with simple cat...,Recommended,68,8.1 hrs on record,Posted: 5 April
1,EARLY ACCESS REVIEWnoo,Recommended,20,19.1 hrs on record,Posted: 5 April
2,EARLY ACCESS REVIEWsigma game,Recommended,26,44.1 hrs on record,Posted: 5 April
3,EARLY ACCESS REVIEWsure,Recommended,21,5.2 hrs on record,Posted: 5 April
4,EARLY ACCESS REVIEWi think this is a fun game ...,Recommended,64,140.7 hrs on record,Posted: 5 April
...,...,...,...,...,...
295,EARLY ACCESS REVIEWLethal Company: A Searing I...,Not Recommended,1661,51.3 hrs on record,Posted: April 5
296,EARLY ACCESS REVIEWhigh impact sexual violence,Recommended,41,151.1 hrs on record,Posted: April 5
297,EARLY ACCESS REVIEWbest game i have played in ...,Recommended,77,95.8 hrs on record,Posted: April 5
298,EARLY ACCESS REVIEWVery fun to play with famil...,Recommended,106,46.8 hrs on record,Posted: April 5


In [33]:
df2.to_csv('LethalCompany.csv', index=False)

# Additional Information

## Craftopia

In [39]:
game_id = 1307550 #change the id for each game, in this case, id is for a game called [Craftopia]
url_template = "https://steamdb.info/app/{}/"
url = url_template.format(game_id)

print(url)

https://steamdb.info/app/1307550/


In [40]:
options = EdgeOptions()
options.use_chromium = True  # Specify using Chromium-based Edge

language = "en-US"
options.add_argument(f"--lang={language}")

# Initialize Edge WebDriver with options
driver = Edge(options=options)

driver.maximize_window()
driver.get(url)

In [41]:
# Extracting the title
title = driver.find_element(By.CSS_SELECTOR, 'h1[itemprop="name"]').text

# Extracting the developer
developer = driver.find_element(By.XPATH, "//td[text()='Developer']/following-sibling::td").text

# Extracting the in-game count
ingame_count = driver.find_element(By.ID, "js-charts-button").find_element(By.TAG_NAME, "div").text

# Extracting tags
tags_elements = driver.find_elements(By.CSS_SELECTOR, ".store-tags a")
tags = [tag.text for tag in tags_elements]

# Extracting categories
categories_elements = driver.find_elements(By.CSS_SELECTOR, ".store-categories a")
categories = [category.text for category in categories_elements]

# Output extracted information
print(f"Title: {title}")
print(f"Developer: {developer}")
print(f"In-Game Count: {ingame_count}")
print(f"Tags: {tags}")
print(f"Categories: {categories}")

# Close the browser
driver.quit()

Title: Craftopia
Developer: Pocketpair
In-Game Count: 400
Tags: ['🌍 Open World', '🔧 Crafting', '🌲 Survival', '🕹️ Multiplayer', '🌎 Open World Survival Craft', '🎲 RPG', '🧱 Sandbox', '🏠 Building', '👥 Online Co-Op', '🗺️ Adventure', '💥 Action', '🐤 Early Access', '🤝 Co-op', '🌸 Anime', '🦹 Character Customization', '🌱 Indie', '🛡️ Hack and Slash', '🤖 Automation', '👤 Singleplayer', '👤 Third Person']
Categories: ['Single-player', 'Online Co-op', 'LAN Co-op', 'Steam Achievements', 'Steam Workshop', 'Remote Play on Tablet', 'Family Sharing', 'Early Access', 'Multi-player', 'Co-op', 'Partial Controller Support']


In [42]:
df_Craftopia = pd.DataFrame({
    "Title": [title],
    "Developer": [developer],
    "In-Game Count": [ingame_count],
    "Tags": [', '.join(tags)],
    "Categories": [', '.join(categories)]
})
df_Craftopia

,Title,Developer,In-Game Count,Tags,Categories
0,Craftopia,Pocketpair,400,"🌍 Open World, 🔧 Crafting, 🌲 Survival, 🕹️ Multi...","Single-player, Online Co-op, LAN Co-op, Steam ..."


In [43]:
df_craftopia = df_craftopia.transpose()
df_craftopia

,0
Title,Craftopia
Developer,Pocketpair
In-Game Count,400
Tags,"🌍 Open World, 🔧 Crafting, 🌲 Survival, 🕹️ Multi..."
Categories,"Single-player, Online Co-op, LAN Co-op, Steam ..."


In [46]:
df_craftopia.to_csv('InfoCraftopia.csv', index=False)

## Palworld

In [44]:
url = url_template.format(1623730)
options = EdgeOptions()
options.use_chromium = True  # Specify using Chromium-based Edge

language = "en-US"
options.add_argument(f"--lang={language}")

# Initialize Edge WebDriver with options
driver = Edge(options=options)

driver.maximize_window()
driver.get(url)

# Extracting the title
title = driver.find_element(By.CSS_SELECTOR, 'h1[itemprop="name"]').text

# Extracting the developer
developer = driver.find_element(By.XPATH, "//td[text()='Developer']/following-sibling::td").text

# Extracting the in-game count
ingame_count = driver.find_element(By.ID, "js-charts-button").find_element(By.TAG_NAME, "div").text

# Extracting tags
tags_elements = driver.find_elements(By.CSS_SELECTOR, ".store-tags a")
tags = [tag.text for tag in tags_elements]

# Extracting categories
categories_elements = driver.find_elements(By.CSS_SELECTOR, ".store-categories a")
categories = [category.text for category in categories_elements]

# Output extracted information
print(f"Title: {title}")
print(f"Developer: {developer}")
print(f"In-Game Count: {ingame_count}")
print(f"Tags: {tags}")
print(f"Categories: {categories}")

# Close the browser
driver.quit()

Title: Palworld
Developer: Pocketpair
In-Game Count: 89,016
Tags: ['🕹️ Multiplayer', '🌍 Open World', '🦎 Creature Collector', '🌲 Survival', '🌎 Open World Survival Craft', '🔧 Crafting', '🤝 Co-op', '🗺️ Adventure', '🧱 Sandbox', '🔫 Third-Person Shooter', '🤖 Automation', '💥 Action', '🐤 Early Access', '🎲 RPG', '🌸 Anime', '🌲 PvE', '🧊 3D', '🌱 Indie', '🎒 Looter Shooter', '🛡️ Hack and Slash']
Categories: ['Single-player', 'Online Co-op', 'Steam Achievements', 'Steam Cloud', 'Family Sharing', 'Early Access', 'Multi-player', 'Co-op', 'Full Controller Support', 'DualShock Controllers (USB Only)', 'DualShock Controllers', 'DualSense Controllers (USB Only)', 'DualSense Controllers']


In [47]:
df_palworld = pd.DataFrame({
    "Title": [title],
    "Developer": [developer],
    "In-Game Count": [ingame_count],
    "Tags": [', '.join(tags)],
    "Categories": [', '.join(categories)]
})
df_palworld

,Title,Developer,In-Game Count,Tags,Categories
0,Palworld,Pocketpair,"89,016","🕹️ Multiplayer, 🌍 Open World, 🦎 Creature Colle...","Single-player, Online Co-op, Steam Achievement..."


In [48]:
df_palworld = df_palworld.transpose()
df_palworld

,0
Title,Palworld
Developer,Pocketpair
In-Game Count,"89,016"
Tags,"🕹️ Multiplayer, 🌍 Open World, 🦎 Creature Colle..."
Categories,"Single-player, Online Co-op, Steam Achievement..."


In [49]:
df_palworld.to_csv('InfoPalworld.csv', index=False)

## Lethal Company

In [54]:
url = url_template.format(1966720)
options = EdgeOptions()
options.use_chromium = True  # Specify using Chromium-based Edge

language = "en-US"
options.add_argument(f"--lang={language}")

# Initialize Edge WebDriver with options
driver = Edge(options=options)

driver.maximize_window()
driver.get(url)

# Extracting the title
title = driver.find_element(By.CSS_SELECTOR, 'h1[itemprop="name"]').text

# Extracting the developer
developer = driver.find_element(By.XPATH, "//td[text()='Developer']/following-sibling::td").text

# Extracting the in-game count
ingame_count = driver.find_element(By.ID, "js-charts-button").find_element(By.TAG_NAME, "div").text

# Extracting tags
tags_elements = driver.find_elements(By.CSS_SELECTOR, ".store-tags a")
tags = [tag.text for tag in tags_elements]

# Extracting categories
categories_elements = driver.find_elements(By.CSS_SELECTOR, ".store-categories a")
categories = [category.text for category in categories_elements]

# Output extracted information
print(f"Title: {title}")
print(f"Developer: {developer}")
print(f"In-Game Count: {ingame_count}")
print(f"Tags: {tags}")
print(f"Categories: {categories}")

# Close the browser
driver.quit()

Title: Lethal Company
Developer: Zeekerss
In-Game Count: 32,729
Tags: ['👥 Online Co-Op', '😱 Horror', '🙌 First-Person', '🤝 Co-op', '🎒 Survival Horror', '🕴 Psychological Horror', '🗺 Exploration', '🚀 Sci-fi', '😃 Funny', '🌲 PvE', '🌫 Atmospheric', '🔀 Procedural Generation', '👽 Aliens', '🕒 Time Management', '🗺️ Adventure', '🏰 Dungeon Crawler', '💥 Action', '🐤 Early Access', '🌄 Action-Adventure', '☠️ Perma Death']
Categories: ['Single-player', 'Online Co-op', 'Family Sharing', 'Early Access', 'Multi-player', 'Co-op', 'Partial Controller Support']


In [55]:
df_lethal = pd.DataFrame({
    "Title": [title],
    "Developer": [developer],
    "In-Game Count": [ingame_count],
    "Tags": [', '.join(tags)],
    "Categories": [', '.join(categories)]
})
df_lethal

,Title,Developer,In-Game Count,Tags,Categories
0,Lethal Company,Zeekerss,"32,729","👥 Online Co-Op, 😱 Horror, 🙌 First-Person, 🤝 Co...","Single-player, Online Co-op, Family Sharing, E..."


In [56]:
df_lethal = df_lethal.transpose()
df_lethal

,0
Title,Lethal Company
Developer,Zeekerss
In-Game Count,"32,729"
Tags,"👥 Online Co-Op, 😱 Horror, 🙌 First-Person, 🤝 Co..."
Categories,"Single-player, Online Co-op, Family Sharing, E..."


In [57]:
df_lethal.to_csv('InfoLethal.csv', index=False)

# Data about Pricing

## Craftopia

In [59]:
game_id = 1307550 #change the id for each game, in this case, id is for a game called [Craftopia]
url_template = "https://steamdb.info/app/{}/"
url = url_template.format(game_id)

options = EdgeOptions()
options.use_chromium = True  # Specify using Chromium-based Edge

language = "en-US"
options.add_argument(f"--lang={language}")

# Initialize Edge WebDriver with options
driver = Edge(options=options)

driver.maximize_window()
driver.get(url)

In [60]:
# Access the prices table
prices_table = driver.find_element(By.CLASS_NAME, "table-prices")

# Find rows within the table
rows = prices_table.find_elements(By.TAG_NAME, "tr")

# List to hold all price data
price_data = []

for row in rows:
    # Extracting data from each row
    cells = row.find_elements(By.TAG_NAME, "td")
    if len(cells) > 4:  # Ensure the row has the number of expected cells
        currency_data = {
            "Currency": cells[0].text,
            "Current Price": cells[1].text,
            "Converted Price": cells[2].text,
            "Lowest Recorded Price": cells[4].text
        }
        price_data.append(currency_data)

# Output extracted information
for data in price_data:
    print(data)

# Close the browser
driver.quit()

{'Currency': 'British Pound', 'Current Price': '£12.59 at -40%', 'Converted Price': '£12.59', 'Lowest Recorded Price': '£11.69 at -40%'}
{'Currency': 'South Asia - USD', 'Current Price': '$5.09 at -40%', 'Converted Price': '£4.02', 'Lowest Recorded Price': '£4.02'}
{'Currency': 'Russian Ruble', 'Current Price': '552 ₽ at -40%', 'Converted Price': '£4.72', 'Lowest Recorded Price': '£2.38'}
{'Currency': 'South African Rand', 'Current Price': 'R 138.00 at -40%', 'Converted Price': '£5.85', 'Lowest Recorded Price': '£3.81'}
{'Currency': 'CIS - U.S. Dollar', 'Current Price': '$7.49 at -40%', 'Converted Price': '£5.92', 'Lowest Recorded Price': '£5.92'}
{'Currency': 'LATAM - U.S. Dollar', 'Current Price': '$7.49 at -40%', 'Converted Price': '£5.92', 'Lowest Recorded Price': '£5.92'}
{'Currency': 'MENA - U.S. Dollar', 'Current Price': '$7.49 at -40%', 'Converted Price': '£5.92', 'Lowest Recorded Price': '£5.92'}
{'Currency': 'Taiwan Dollar', 'Current Price': 'NT$ 244 at -40%', 'Converted Pric

In [61]:
Craftopia_prices = pd.DataFrame(price_data)
Craftopia_prices

,Currency,Current Price,Converted Price,Lowest Recorded Price
0,British Pound,£12.59 at -40%,£12.59,£11.69 at -40%
1,South Asia - USD,$5.09 at -40%,£4.02,£4.02
2,Russian Ruble,552 ₽ at -40%,£4.72,£2.38
3,South African Rand,R 138.00 at -40%,£5.85,£3.81
4,CIS - U.S. Dollar,$7.49 at -40%,£5.92,£5.92
5,LATAM - U.S. Dollar,$7.49 at -40%,£5.92,£5.92
6,MENA - U.S. Dollar,$7.49 at -40%,£5.92,£5.92
7,Taiwan Dollar,NT$ 244 at -40%,£6.01,£5.57
8,Chinese Yuan,¥ 55.20 at -40%,£6.04,£5.25
9,Malaysian Ringgit,RM36.60 at -40%,£6.10,£4.40


In [64]:
Craftopia_prices.to_csv('Craftopia_pricing.csv', index=False)

## Palworld

In [62]:
url = url_template.format(1623730)
options = EdgeOptions()
options.use_chromium = True  # Specify using Chromium-based Edge

language = "en-US"
options.add_argument(f"--lang={language}")

# Initialize Edge WebDriver with options
driver = Edge(options=options)

driver.maximize_window()
driver.get(url)

# Access the prices table
prices_table = driver.find_element(By.CLASS_NAME, "table-prices")

# Find rows within the table
rows = prices_table.find_elements(By.TAG_NAME, "tr")

# List to hold all price data
price_data = []

for row in rows:
    # Extracting data from each row
    cells = row.find_elements(By.TAG_NAME, "td")
    if len(cells) > 4:  # Ensure the row has the number of expected cells
        currency_data = {
            "Currency": cells[0].text,
            "Current Price": cells[1].text,
            "Converted Price": cells[2].text,
            "Lowest Recorded Price": cells[4].text
        }
        price_data.append(currency_data)

# Output extracted information
for data in price_data:
    print(data)

# Close the browser
driver.quit()

{'Currency': 'British Pound', 'Current Price': '£22.49 at -10%', 'Converted Price': '£22.49', 'Lowest Recorded Price': '£22.49 at -10%'}
{'Currency': 'South Asia - USD', 'Current Price': '$9.44 at -10%', 'Converted Price': '£7.47', 'Lowest Recorded Price': '£7.47'}
{'Currency': 'Russian Ruble', 'Current Price': '990 ₽ at -10%', 'Converted Price': '£8.47', 'Lowest Recorded Price': '£8.47'}
{'Currency': 'South African Rand', 'Current Price': 'R 243.00 at -10%', 'Converted Price': '£10.31', 'Lowest Recorded Price': '£10.31'}
{'Currency': 'CIS - U.S. Dollar', 'Current Price': '$13.04 at -10%', 'Converted Price': '£10.32', 'Lowest Recorded Price': '£10.32'}
{'Currency': 'Chinese Yuan', 'Current Price': '¥ 97.20 at -10%', 'Converted Price': '£10.63', 'Lowest Recorded Price': '£10.63'}
{'Currency': 'LATAM - U.S. Dollar', 'Current Price': '$13.49 at -10%', 'Converted Price': '£10.68', 'Lowest Recorded Price': '£10.68'}
{'Currency': 'MENA - U.S. Dollar', 'Current Price': '$13.49 at -10%', 'Conv

In [63]:
Palworld_prices = pd.DataFrame(price_data)
Palworld_prices

,Currency,Current Price,Converted Price,Lowest Recorded Price
0,British Pound,£22.49 at -10%,£22.49,£22.49 at -10%
1,South Asia - USD,$9.44 at -10%,£7.47,£7.47
2,Russian Ruble,990 ₽ at -10%,£8.47,£8.47
3,South African Rand,R 243.00 at -10%,£10.31,£10.31
4,CIS - U.S. Dollar,$13.04 at -10%,£10.32,£10.32
5,Chinese Yuan,¥ 97.20 at -10%,£10.63,£10.63
6,LATAM - U.S. Dollar,$13.49 at -10%,£10.68,£10.68
7,MENA - U.S. Dollar,$13.49 at -10%,£10.68,£10.68
8,Malaysian Ringgit,RM64.80 at -10%,£10.80,£10.80
9,Taiwan Dollar,NT$ 439 at -10%,£10.82,£10.82


In [65]:
Palworld_prices.to_csv('Palworld_pricing.csv', index=False)

## Lethal Company

In [66]:
url = url_template.format(1966720)
options = EdgeOptions()
options.use_chromium = True  # Specify using Chromium-based Edge

language = "en-US"
options.add_argument(f"--lang={language}")

# Initialize Edge WebDriver with options
driver = Edge(options=options)

driver.maximize_window()
driver.get(url)

# Access the prices table
prices_table = driver.find_element(By.CLASS_NAME, "table-prices")

# Find rows within the table
rows = prices_table.find_elements(By.TAG_NAME, "tr")

# List to hold all price data
price_data = []

for row in rows:
    # Extracting data from each row
    cells = row.find_elements(By.TAG_NAME, "td")
    if len(cells) > 4:  # Ensure the row has the number of expected cells
        currency_data = {
            "Currency": cells[0].text,
            "Current Price": cells[1].text,
            "Converted Price": cells[2].text,
            "Lowest Recorded Price": cells[4].text
        }
        price_data.append(currency_data)

# Output extracted information
for data in price_data:
    print(data)

# Close the browser
driver.quit()

{'Currency': 'British Pound', 'Current Price': '£8.50', 'Converted Price': '£8.50', 'Lowest Recorded Price': '£5.95 at -30%'}
{'Currency': 'Russian Ruble', 'Current Price': '385 ₽', 'Converted Price': '£3.29', 'Lowest Recorded Price': '£2.30'}
{'Currency': 'South African Rand', 'Current Price': 'R 100.00', 'Converted Price': '£4.24', 'Lowest Recorded Price': '£2.97'}
{'Currency': 'South Asia - USD', 'Current Price': '$5.49', 'Converted Price': '£4.34', 'Lowest Recorded Price': '£3.04'}
{'Currency': 'Malaysian Ringgit', 'Current Price': 'RM26.75', 'Converted Price': '£4.46', 'Lowest Recorded Price': '£3.12'}
{'Currency': 'Vietnamese Dong', 'Current Price': '142000₫', 'Converted Price': '£4.50', 'Lowest Recorded Price': '£3.15'}
{'Currency': 'Indonesian Rupiah', 'Current Price': 'Rp 90999', 'Converted Price': '£4.53', 'Lowest Recorded Price': '£3.17'}
{'Currency': 'Ukrainian Hryvnia', 'Current Price': '225₴', 'Converted Price': '£4.55', 'Lowest Recorded Price': '£3.18'}
{'Currency': 'Ind

In [69]:
Lethal_prices = pd.DataFrame(price_data)
Lethal_prices

,Currency,Current Price,Converted Price,Lowest Recorded Price
0,British Pound,£8.50,£8.50,£5.95 at -30%
1,Russian Ruble,385 ₽,£3.29,£2.30
2,South African Rand,R 100.00,£4.24,£2.97
3,South Asia - USD,$5.49,£4.34,£3.04
4,Malaysian Ringgit,RM26.75,£4.46,£3.12
5,Vietnamese Dong,142000₫,£4.50,£3.15
6,Indonesian Rupiah,Rp 90999,£4.53,£3.17
7,Ukrainian Hryvnia,225₴,£4.55,£3.18
8,Indian Rupee,₹ 480,£4.56,£3.19
9,LATAM - U.S. Dollar,$5.79,£4.58,£3.20


In [68]:
Lethal_prices.to_csv('Lethal_pricing.csv', index=False)